In [2]:
#Testing Pyspark Installation
import findspark

findspark.init()
findspark.find()

import pyspark
findspark.find()

'C:\\spark'

In [3]:
#intialize Spark Context
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('Inter3').getOrCreate()

In [4]:
import pandas as pd
from pyspark.sql import *
from pyspark.sql.types import *

#main1=pd.read_excel('/Users/Asus/Documents/InternCsv/forcast_july/july3.xls', index_col=0)  
#main1=pd.read_excel('/Users/Asus/Documents/InternCsv/forcast_july/july4.xls', index_col=0)  

main1=spark.read.option('header','false').csv('/Users/Asus/Documents/InternCsv/forcast_july/july3.csv')
main2=spark.read.option('header','false').csv('/Users/Asus/Documents/InternCsv/forcast_july/july4.csv')

In [8]:
import pyspark
from pyspark.sql import SparkSession

#main1.printSchema()
from pyspark.sql.functions import col
main1 = main1.select(col("_c0").alias("Starting_Time1"), col("_c1").alias("Starting_Time2"),col("_c2").alias("Starting_Time3"))
main1.show(6)
#main2.printSchema()
main2 = main2.select(col("_c0").alias("Starting_Time1"), col("_c1").alias("Starting_Time2"),col("_c2").alias("Starting_Time3"))
main2.show(6)

+--------------------+--------------+--------------+
|      Starting_Time1|Starting_Time2|Starting_Time3|
+--------------------+--------------+--------------+
|3.    Day Ahead E...|          null|          null|
|                Date|     03-Jul-20|          null|
|                null|          0:00|          0:30|
|            WPS     |            23|            10|
|           Cany     |             0|             0|
|         Olax       |            10|            10|
+--------------------+--------------+--------------+
only showing top 6 rows

+--------------------+--------------+--------------+
|      Starting_Time1|Starting_Time2|Starting_Time3|
+--------------------+--------------+--------------+
|3.    Day Ahead E...|          null|          null|
|                Date|     04-Jul-20|          null|
|                null|          0:00|          0:30|
|            WPS     |            23|            23|
|           Cany     |            11|            11|
|         Olax       

In [9]:
main3=main1.join(main2,main1.Starting_Time1 ==  main1.Starting_Time1,"inner") \
    .show(truncate=False)

+----------------------------------------------------+--------------+--------------+----------------------------------------------------+--------------+--------------+
|Starting_Time1                                      |Starting_Time2|Starting_Time3|Starting_Time1                                      |Starting_Time2|Starting_Time3|
+----------------------------------------------------+--------------+--------------+----------------------------------------------------+--------------+--------------+
|3.    Day Ahead Economic Dispatch for next day in MW|null          |null          |3.    Day Ahead Economic Dispatch for next day in MW|null          |null          |
|3.    Day Ahead Economic Dispatch for next day in MW|null          |null          |Date                                                |04-Jul-20     |null          |
|3.    Day Ahead Economic Dispatch for next day in MW|null          |null          |null                                                |0:00          |0:30    

In [27]:
import pyspark.sql.types
from datetime import timedelta, datetime
import time
import random
from pyspark.sql.functions import udf,when

Inter1=main1.withColumn("Total_Thermal_Comparision",
       when((col("Starting_Time2") >= "20") & (col("Starting_Time3") >= "40") , "Success").otherwise("03-Jul-20"))
Inter1.show(100)

+--------------------+--------------------+--------------+-------------------------+
|      Starting_Time1|      Starting_Time2|Starting_Time3|Total_Thermal_Comparision|
+--------------------+--------------------+--------------+-------------------------+
|3.    Day Ahead E...|                null|          null|                03-Jul-20|
|                Date|           03-Jul-20|          null|                03-Jul-20|
|                null|                0:00|          0:30|                03-Jul-20|
|            WPS     |                  23|            10|                03-Jul-20|
|           Cany     |                   0|             0|                03-Jul-20|
|         Olax       |                  10|            10|                03-Jul-20|
|                Nlax|                  20|            20|                03-Jul-20|
|                Polp|                  22|             6|                  Success|
|                UKPS|                   0|             0|       

In [26]:
Inter2=main2.withColumn("Total_Thermal_Comparision",
       when((col("Starting_Time2") >= "20") & (col("Starting_Time3") >= "40") , "Success").otherwise("Need Improvement"))
Inter2.show(100)

+--------------------+--------------------+--------------+-------------------------+
|      Starting_Time1|      Starting_Time2|Starting_Time3|Total_Thermal_Comparision|
+--------------------+--------------------+--------------+-------------------------+
|3.    Day Ahead E...|                null|          null|         Need Improvement|
|                Date|           04-Jul-20|          null|         Need Improvement|
|                null|                0:00|          0:30|         Need Improvement|
|            WPS     |                  23|            23|         Need Improvement|
|           Cany     |                  11|            11|         Need Improvement|
|         Olax       |                  10|            10|         Need Improvement|
|                Nlax|                  22|            22|         Need Improvement|
|                Polp|                  19|            18|         Need Improvement|
|                UKPS|                   0|             0|       

In [28]:
from pyspark.sql.functions import col
#Total Generation in 03-Jul-20
#Filter multiple condition
Gen1=Inter1.filter((Inter1.Starting_Time1 == "Total Generation") )
Gen1.show()

+----------------+--------------+--------------+-------------------------+
|  Starting_Time1|Starting_Time2|Starting_Time3|Total_Thermal_Comparision|
+----------------+--------------+--------------+-------------------------+
|Total Generation|          1434|          1393|                03-Jul-20|
+----------------+--------------+--------------+-------------------------+



In [29]:
#Total Generation in 04-Jul-20
Gen2=Inter2.filter((Inter2.Starting_Time1 == "Total Generation") )
Gen2.show()

+----------------+--------------+--------------+-------------------------+
|  Starting_Time1|Starting_Time2|Starting_Time3|Total_Thermal_Comparision|
+----------------+--------------+--------------+-------------------------+
|Total Generation|          1302|          1257|         Need Improvement|
+----------------+--------------+--------------+-------------------------+



In [30]:
Man=Gen1.union(Gen2) 
Man.show()

+----------------+--------------+--------------+-------------------------+
|  Starting_Time1|Starting_Time2|Starting_Time3|Total_Thermal_Comparision|
+----------------+--------------+--------------+-------------------------+
|Total Generation|          1434|          1393|                03-Jul-20|
|Total Generation|          1302|          1257|         Need Improvement|
+----------------+--------------+--------------+-------------------------+



In [39]:
Man.withColumn("Success Rate %",
       when((col("Total_Thermal_Comparision")==" 03-Jul-20" ) | (col("Starting_Time3") > "1300") , "First Day is Success").otherwise("Has to Improve")) \
.show()


+----------------+--------------+--------------+-------------------------+--------------------+
|  Starting_Time1|Starting_Time2|Starting_Time3|Total_Thermal_Comparision|      Success Rate %|
+----------------+--------------+--------------+-------------------------+--------------------+
|Total Generation|          1434|          1393|                03-Jul-20|First Day is Success|
|Total Generation|          1302|          1257|         Need Improvement|      Has to Improve|
+----------------+--------------+--------------+-------------------------+--------------------+

